## Libraries used :

In [1]:
#Dataframe 
import pandas as pd

#Calculus 
import numpy as np 
from numpy import log as ln

#Date 
from datetime import date

#Yfinance and scraping 
import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as pdr

import yahoo_fin.stock_info as si
from yahoo_fin import options 
import math 
from cmath import sqrt

In [2]:
#Settings df output 10 is by default, write None to get all values 
pd.set_option("display.max_rows", 10, "display.max_columns", 10)

## 1) Scraping data from yahoo finance :

In [6]:
#Get date for today
today = date.today()

#Choose a period of time 
start_date= '2022-09-13'
end_date= '2018-01-01'

#Check busday count
bourse_day = np.busday_count(start_date, today,weekmask='1111100')


#Do a len of the df to check if its true
len_check = len(df_unfilt['adjclose'])
N = len_check
F = 1

#Print the results 
print("Date today is :",today)
print("")
print("For the following dates you will get",bourse_day, "data")
print("")
print("Checking with the len there is",len_check, "data")

Date today is : 2022-10-20

For the following dates you will get 27 data

Checking with the len there is 27 data


In [7]:
#Get data from yfinance 
df_unfilt = si.get_data("aapl",start_date,today)
df = df_unfilt[['open','high','low','adjclose']]
df.reset_index(drop=False)
df
#Round the dataframe
# df.apply(np.round)

,open,high,low,adjclose
2022-09-13,159.899994,160.539993,153.369995,153.839996
2022-09-14,154.789993,157.100006,153.610001,155.309998
2022-09-15,154.649994,155.240005,151.380005,152.369995
2022-09-16,151.210007,151.350006,148.369995,150.699997
2022-09-19,149.309998,154.559998,149.100006,154.479996
...,...,...,...,...
2022-10-13,134.990005,143.589996,134.369995,142.990005
2022-10-14,144.309998,144.520004,138.190002,138.380005
2022-10-17,141.070007,142.899994,140.270004,142.410004
2022-10-18,145.490005,146.699997,140.610001,143.750000


In [8]:
#Few tests 
opening = df.iloc[0,0]
high = df.iloc[0,1]
low = df.iloc[0,2]
close = df.iloc[0,3]

print('open = ',opening)
print('high = ',high)
print('low = ',low)
print('close = ',close)

open =  159.89999389648438
high =  160.5399932861328
low =  153.3699951171875
close =  153.83999633789062


## 2) Rogers-Satchell's volatility calculus  : 

In [9]:
#Construction of Roger's Vol 
x = 0
length = len_check-1
L = []

while x <= length:
    vol = (np.log(df.iloc[(0+x),1]/df.iloc[(0+x),3])*
           np.log(df.iloc[(0+x),1]/df.iloc[(0+x),0])) + (np.log(df.iloc[(0+x),2]/df.iloc[(0+x),3])* 
                                                         np.log(df.iloc[(0+x),2]/df.iloc[(0+x),0]))
    
    L.append(vol)
    print(vol)
    x=x+1

rogers_vol = np.sqrt(F/len_check) * np.sqrt(sum(L))
print("")
print("La volatilité de Rogers-Satchell est égale à : ",rogers_vol)

0.0002978655707313606
0.0002539759049937757
0.00021036502159743996
0.00029942523643939234
6.778072864659918e-05
0.00027664059873507474
0.000303457888250114
0.00027027050756664965
0.00023225970195509403
0.0005347838759079068
0.00046999126658888406
0.0007569399730356809
0.0006048082820419699
0.00047999396286999306
0.0002782058595196325
7.417881035616904e-05
0.00032456078422530806
0.00017575777551251344
0.00018371249024155904
0.0002843426275366466
0.00024060069359740276
0.00013670064920014944
0.0005448501619150544
0.0001226735321576525
0.00013037920637112458
0.0009217458491002962
0.0001938971053013127

La volatilité de Rogers-Satchell est égale à :  0.017919742954913616


## 3) Overgnight volatility calculus : 

There is several steps to get this Volatility :

- Step 1 : Get the average of Log(opening / opening - 1)
- Step 2 : Get the Log(open / closing-1) - Step 1
- Step 3 : Do the sum of all parameters 
- Step 4 : Multiply by 1/N-1 the result 

In [12]:
#Create special df for overnight volatility cause of the O(-1) 

#Get date for today
today = date.today()

#Choose a period of time 
start_date_overnight = '2022-09-12'
end_date = '2018-01-01'

#Check busday count
bourse_day_overnight = np.busday_count(start_date_overnight, today,weekmask='1111100')


#Do a len of the df to check if its true
len_check_over = len(df_unfilt_over['adjclose'])
N = len_check
F = 1

#Print the results 
print("For the following dates you will get",bourse_day_overnight, "data")
print("")
print("Checking with the len there is",len_check_over, "data")


For the following dates you will get 28 data

Checking with the len there is 28 data


In [13]:
#Get data from yfinance 
df_unfilt_over = si.get_data("aapl",start_date_overnight,today)
df_over = df_unfilt_over[['open','high','low','adjclose']]
df_over.sort_index()
df_over.reset_index(drop=True,inplace=True)
df_over

#Do a len of the df to check if its true
len_check = len(df_over['adjclose'])

#Round the dataframe
# df.apply(np.round)

df_over

,open,high,low,adjclose
0,159.589996,164.259995,159.300003,163.429993
1,159.899994,160.539993,153.369995,153.839996
2,154.789993,157.100006,153.610001,155.309998
3,154.649994,155.240005,151.380005,152.369995
4,151.210007,151.350006,148.369995,150.699997
...,...,...,...,...
23,134.990005,143.589996,134.369995,142.990005
24,144.309998,144.520004,138.190002,138.380005
25,141.070007,142.899994,140.270004,142.410004
26,145.490005,146.699997,140.610001,143.750000


In [14]:
#Few tests 
opening = df_over.iloc[0,0]
high = df_over.iloc[0,1]
low = df_over.iloc[0,2]
close = df_over.iloc[0,3]

print('open = ',opening)
print('high = ',high)
print('low = ',low)
print('close = ',close)

open =  159.58999633789062
high =  164.25999450683594
low =  159.3000030517578
close =  163.42999267578125


In [15]:
# Parameters 
k = 0.34/(1.34+(N+1/N-1))
print('Value of K is :',k)

Value of K is : 0.01198207722901878


In [16]:
#Overnight volatility (2 steps, average month while loop and day to day)

"""STEP 1"""
x = 0
length = 25-2
L2 = []

while x <= length:
    vol_night_mean = np.log(df_over.iloc[(0+x),0]/df_over.iloc[(1+x),3])
            

    L2.append(vol_night_mean)
    print(vol_night_mean)
   
    x=x+1

terme_2_night = np.mean(L2)    
   
    
# Add final parameters 1/N-1

0.03669492598824863
0.029125477806727765
0.015757574776245783
0.025873374282539455
-0.02139496693768579
-0.049583955650998
-0.0020839244928075795
0.02967188020157746
0.0128796220673403
0.0027818150116111807
-0.013934210308629718
0.01916913172868535
0.03557526503272973
0.05558947128155514
-0.008247311305216816
-0.05551704433971686
-0.00940195068127194
-0.009395477415817267
0.04001934051754023
0.014984696422827392
0.010307911404631706
0.011213441343097551
-0.027365953212066963
-0.024802817042347332


In [17]:
len(L2)

24

In [19]:
"""STEP 2"""
L3 = []
x = 0
length = 25-2

while x <= length:
    vol_night_final = (np.log(df_over.iloc[(0+x),0]/df_over.iloc[(1+x),3]) - terme_2_night)**2
    
    L3.append(vol_night_final)
    print(vol_night_final)
    x=x+1
    
overnight_pre_end = np.sum(L3)   
overnight_volatility = overnight_pre_end /25
print('overnight_volatility = ',overnight_volatility)

0.0010100793873873833
0.0005862353721863706
0.00011760090124517429
0.00043932975051222037
0.0006921185875751778
0.002969937778171998
4.8959469230084e-05
0.000612993242936621
6.346420154990926e-05
4.542716088316056e-06
0.00035522411591537594
0.000203232163873779
0.0009401634675516279
0.0025680865129231374
0.00017319852753295995
0.003651811996016263
0.00020492296222135607
0.00020473767284458474
0.0012324425165846202
0.00010143544601060102
2.9103128511113886e-05
3.969329484003638e-05
0.001041942431450589
0.0008830404715011299
overnight_volatility =  0.0007269718445864171


## 4) Open to close volatility calculus : 

There is also several steps to get this Volatility :

- Step 1 : Get the average of Log(closing / opening) : we can re-use the first df  
- Step 2 : Get the Log(open / closing) - Step 1(the average)
- Step 3 : Do the sum of all parameters 
- Step 4 : Multiply by 1/N-1 the result 

In [39]:
#Construction of Open-to-Close Volatility 

"""STEP 1"""
x = 0
length = len_check-2
L5 = []

while x <= length:
    vol_otc_mean = np.log(df.iloc[(0+x),3]/df.iloc[(0+x),0])
            

    L5.append(vol_otc_mean)
    print(vol_otc_mean)
   
    x=x+1

terme_2_otc = np.mean(L5)
print("")
print("terme_2_otc = ",terme_2_otc)

-0.038635504193217526
0.0033537876506024187
-0.014852718228936974
-0.00337855814362369
0.03403994527589461
0.022559771693395427
-0.023276271988238305
0.0023597323358657382
-0.00503952911886432
0.007389446006694293
-0.006436875994017336
0.014791159667955781
-0.02508975153792542
-0.022041838963953526
0.03021672332733204
0.007350751377682741
0.016043216425232112
-0.0026095667573284203
-0.017337567905184505
0.0
-0.006597830899103154
-0.005694386248475066
0.05757399384732306
-0.04196018717399579
0.009453973941142283
-0.012031713806136608
0.015199018875134356

terme_2_otc =  -0.00017225113091651008


In [40]:
"""STEP 2"""
L6 = []
x = 0
length = len_check-2

while x <= length:
    vol_otc = (np.log(df.iloc[(0+x),3]/df.iloc[(0+x),0]) - terme_2_otc)**2
    
    L6.append(vol_otc)
    print(vol_otc)
    x=x+1
    
otc_pre_end = np.sum(L6)  
otc_volatility = overnight_pre_end /length 
print('overnight_volatility = ',otc_volatility)

0.0014794218361346085
1.2432949488775493e-05
0.0002155161142160614
1.0280404659735238e-05
0.0011704743829782198
0.0005167448616850388
0.0005337957797755606
6.4109402760586535e-06
2.3690395011961283e-05
5.717926360095141e-05
3.9245524675381044e-05
0.0002239036627358079
0.0006208818265332893
0.0004782788719869207
0.0009234897686240827
5.6595566744390624e-05
0.0002629413880645085
5.940507462751681e-06
0.00029464809996096626
2.9670452102016697e-08
4.1288075357329516e-05
3.0493976256573446e-05
0.003334628809086859
0.0017462315987404845
9.266420913793332e-05
0.00014064685494493865
0.000236275941598919
overnight_volatility =  0.0004828512030842389


## 5) Get Yang-Zhang Volatility :

In [43]:
#Remind all results 

print("rogers_vol =",rogers_vol)
print("")
print("overnight_volatility =",overnight_volatility)
print("")
print("otc_volatility =",otc_volatility)
print("")
print("k value is :",k)

rogers_vol = 0.017919742954913616

overnight_volatility = 0.0004828512030842389

otc_volatility = 0.0004828512030842389

k value is : 0.01198207722901878


In [44]:
#Finally

yang_zhang_volatility = np.sqrt(F) * np.sqrt(overnight_volatility**2 + k*otc_volatility**2 + (1+k)*rogers_vol**2)
print("yang_zhang_volatility is :",yang_zhang_volatility)

yang_zhang_volatility is : 0.01803332407961243


# 6) Other approach, keep data in a dataframe :

In [73]:
df

,open,high,low,adjclose
2022-09-13,159.899994,160.539993,153.369995,153.839996
2022-09-14,154.789993,157.100006,153.610001,155.309998
2022-09-15,154.649994,155.240005,151.380005,152.369995
2022-09-16,151.210007,151.350006,148.369995,150.699997
2022-09-19,149.309998,154.559998,149.100006,154.479996
...,...,...,...,...
2022-10-13,134.990005,143.589996,134.369995,142.990005
2022-10-14,144.309998,144.520004,138.190002,138.380005
2022-10-17,141.070007,142.899994,140.270004,142.410004
2022-10-18,145.490005,146.699997,140.610001,143.750000


In [ ]:
#Create column for robert satchers volatility 



In [74]:
df_over

,open,high,low,adjclose
0,159.589996,164.259995,159.300003,163.429993
1,159.899994,160.539993,153.369995,153.839996
2,154.789993,157.100006,153.610001,155.309998
3,154.649994,155.240005,151.380005,152.369995
4,151.210007,151.350006,148.369995,150.699997
...,...,...,...,...
23,134.990005,143.589996,134.369995,142.990005
24,144.309998,144.520004,138.190002,138.380005
25,141.070007,142.899994,140.270004,142.410004
26,145.490005,146.699997,140.610001,143.750000
